# Modeliranje časovnih vrst

Modeliranje časovnih vrst je pomeben del ekonomskih modelov, borznega posredništva, analize časovnih meritev v fiziki, biologiji, kemiji, ipd.


Podatki v obliki časovnih vrst se od dosedanjih scenarijev razlikujejo po pomebni lastnosti: vzorci <i>niso</i> neodvisni med seboj. Podatke predstavlja vektor časovnih točk (ki niso nujno enako oddaljene):

$$ \mathbf{t} = (t_1, t_2, ..., t_n) $$

Običajno nas zanima <i>funkcija</i> oz. <i>signal</i> v vsaki časovni točki.

$$ x(\mathbf{t}) = (x(t_1), x(t_2), ..., x(t_n)) $$ 

In [1]:
import sys
import mlpy

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import scipy
import os
import GPy

ModuleNotFoundError: No module named 'mlpy'

## Primerjava časovnih vrst

Oglejmo si preprost primer dveh podobnih signalov $x(\mathbf{t})$ in $y(\mathbf{t})$.

In [2]:
resolution = 100
t = np.linspace(-5, 5, resolution)
x = np.sin(t)    * np.cos(2*t) + 0.2*np.random.rand(1, resolution).ravel()
y = -np.sin(t-2) * np.cos(2*t + 4) + 0.2*np.random.rand(1, resolution).ravel()

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(t.reshape((len(t), 1)), x)
z = model.predict(t.reshape((len(t), 1)))

plt.figure()
plt.plot(x, "b.-", label="$x(t)$")
# plt.plot(z, "r-", label="$x(t)$")
plt.plot(y, "r.-", label="$y(t)$")
plt.gca().set_xticklabels(np.linspace(-5, 5, 6))
plt.xlabel("t")
plt.legend()
plt.show()

NameError: name 'np' is not defined

Opazimo, da sta si signala zelo podobna, vendar so vrhovi zelo oddaljeni med seboj. Kako izmeriti to razdaljo (npr. za potrebe hierarhičnega gručenja)? Evklidska razdalja vrne navidez zelo visoko vrednost.

In [3]:
np.linalg.norm(x[:-2]-x[2:], ord=2)

NameError: name 'np' is not defined

Korelacija med signaloma je nizka oz. celo obratna.

In [4]:
scipy.stats.pearsonr(x, y)[0]

NameError: name 'scipy' is not defined

### Dinamična poravnava signalov



V splošnem imamo dva različno dolga signala:

$$ x(\mathbf{t}) = (x(t_1), x(t_2), ..., x(t_n)) $$ 
$$ y(\mathbf{t}) = (y(t_1), y(t_2), ..., y(t_m)) $$ 

Dinamična poravnava signalov (ang. <i>Dynamic time warping</i>, DTW) je algoritem dinamičnega programiranja, ki poišče signala $x_w(\mathbf{t})$ in $y_w(\mathbf{t})$, tako, da je razdalja med vrednostmi signalov $|x_w(t_k) - y_w(t_k)|$ čim manjša. Dovoljeno je lokalno raztezanje in krčenje obeh signalov.

Algoritem DTW sestavi matriko $\mathbf{W}$ velikosti $m \times n$ ki hrani razdalje, tako da 

$$w_{ij} = |x(t_i) - y(t_j)|$$

Cilj algoritma DTW je iskanje <b>poti</b> dolžine $max(m, n)$, ki gre iz levega spodnjega v desni zgornji kot matrike $\mathbf{W}$, tako da zmanjšamo skupno razdaljo

$$ \text{min} \sum_{k} \sqrt{w_k} $$

    

Rezultat algoritma je matrika poravnave, optimalna pot in skupna razdalja med signaloma.

In [5]:
dist, cost, path = mlpy.dtw_std(x, y, dist_only=False)
print("Oddaljenost med x in y", dist)

NameError: name 'mlpy' is not defined

In [6]:
fig = plt.figure(2)
ax = fig.add_subplot(111)
plot1 = plt.imshow(cost.T, origin='lower', cmap=cm.gray, interpolation='nearest')
plot2 = plt.plot(path[0], path[1], 'y', label="Pot w_k")
xlim = ax.set_xlim((-0.5, cost.shape[0]-0.5))
ylim = ax.set_ylim((-0.5, cost.shape[1]-0.5))
plt.xlabel("Indeks $y$")
plt.ylabel("Indeks $x$")
plt.title("Optimalna matrika poravnave",)
plt.legend( loc=4)
plt.show()

NameError: name 'plt' is not defined

Poravnana signala dobimo s poravnavo vrednosti na ustreznih mestih v zaporedju.

In [7]:
xw = x[path[0]]
yw = y[path[1]]

NameError: name 'x' is not defined

Korelacija med signaloma je bistveno večja!

In [8]:
scipy.stats.pearsonr(xw, yw)[0]

NameError: name 'scipy' is not defined

Oba signala sta lokalno deformirana.

In [9]:
plt.figure()
plt.plot(xw, label="$x_w(t)$")
plt.plot(yw, label="$y_w(t)$")
plt.legend()
plt.show()

NameError: name 'plt' is not defined

<font color="green"><b>Naredi sam/a.</b></font> Poišči slovenske občine s podobnimi trendi spreminjanja gostote prebivalstva. Oglej si trende nekaj najpodobnejših krajev.

In [10]:
x = np.loadtxt("data/ages/Maribor_starost-20-24_let.txt")
y = np.loadtxt("data/ages/Ljubljana_starost-20-24_let.txt")

NameError: name 'np' is not defined

In [11]:
# ... load all cities and store them into a matrix
import glob
labels = []
X = []

for f in glob.glob("data/ages/*_starost-20-24_let.txt"):
    city = os.path.basename(f).split("_")[0]
    labels.append(city)
    data = np.loadtxt(f)
    d = scipy.stats.zscore(data)
    X.append(d)
X = np.array(X)
X.shape

NameError: name 'os' is not defined

<font color="green"><b>Namig.</b></font> Uporabi hierarhično razvšanje, kjer
namesto funkcije za razdaljo (`sch.linkage(metric=...)`) podaš razdaljo izmerjeno po DTW. 

In [12]:
import scipy.cluster.hierarchy as sch
# TODO: your code here

<br/>
<br/>
<br/>
<br/>
<br/>
## Neparametrična regresija ali napovedovanje trendov

### Gaussovi procesi

Gaussovi procesi so paradni konj družine modelov, ki ji pravimo <i>neparametrična regresija</i>. Napovedni model tokrat ne bo predstavljen kot vektor uteži, temveč bo vsa informacija za napovedovanje vsebovana v učnem vzorcu. Prednosti pristopa sta:
* predpostavka, da so primeri neodvisni med seboj ne drži več,
* model se posodobi, ko se pojavijo novi primeri.


<br>

Glana predpostavka je naslednja. Funkcija $x(\mathbf{t})$ je porazdeljena po multivariatni normalni porazdelitvi. To ne pomeni, da je vsaka vrednost $(x(t_i))$ porazdeljena normalno, temveč da celoten vektor $x(\mathbf{t})$ prihaja iz <i> skupne noramalne porazdelitve </i>, kjer so posamezne vrednosti $(x(t_i))$ lahko odvisne med sabo!

Torej:


$$ x(\mathbf{t}) \sim \mathcal{N}(m(\mathbf{t}), k(\mathbf{t}, \mathbf{t})) $$ 

Funkcija $m(\mathbf{t})$ je funkcija <i>povprečja</i>, funkcija $k(\mathbf{t}, \mathbf{t})$ pa funkcija <i>kovariance</i>. Večinoma funkcijo povprečja nastavimo na 0, na obliko modela pa bistveno vpliva struktura kovariance. Zapišemo

$$ x(\mathbf{t}) \sim \mathcal{N}(\mathbf{0}, k(\mathbf{t}, \mathbf{t})) $$ 

V praksi to pomeni, da za vsak končen učni vzorec $(x(t_1), x(t_2), ..., x(t_n))$ lahko <i>statistično sklepamo</i> o vsaki drugi časovni točki. Ob predpostavki normalne porazdelitve tako lahko analitično izračunamo naslednjo pogojno verjetnost

$$ p(x(t_*) |  x(t_1), x(t_2), ..., x(t_n)) $$

Za vsako časovno točko $t_*$. Kje je torej skrita informacija o podobnosti med primeri? V <b>kovariančni funkciji</b>!

#### Primer

Oglejmo si spreminjanje bruto državnega proizvoda v Združenih državah amerike med leti 1970 in 2012.

In [13]:
data = np.genfromtxt("data/GDP-USD-countries.csv", delimiter=",")

i = 205
x = data[0, 1:]
y = data[i, 1:]
n = len(x)

plt.figure()
plt.plot(x, y)
plt.show()

NameError: name 'np' is not defined

Uporabili bomo tipično funkcijo kovariance, eksponentno-kvadratno funkcijo (ang. "Exponentiated-quadratic" oz. "RBF"), dano z izrazom

$$ k(t, t') = exp\{-\frac{\|t-t'\|^2}{2\ell^2}\} $$

kjer parametru $\ell$ pravimo dolžina vpliva (ang. <i>lengthscale</i>).

In [14]:
resolution = 10

t = np.linspace(-5, 5, resolution)
x = np.sin(t)    * np.cos(2*t) + 0.2*np.random.rand(1, resolution).ravel()

t = t.reshape((len(t), 1))[0::1]
x = x.reshape((len(x), 1))[0::1]

x = x - x.mean()

# Gaussian kernel, RBF, sq exp, exponentiated quadratic, stationary kernel
kernel = GPy.kern.RBF(input_dim=1, lengthscale=1)
model = GPy.models.GPRegression(t, x, kernel, noise_var=0.1) # noise_var=10.0

# model.optimize(messages=True)
model.plot(lower=5, upper=95)
plt.gca().set_xlabel("t")
plt.gca().set_ylabel("GDP($)")
plt.xlim(-5, 10)
plt.show()

NameError: name 'np' is not defined

Dobimo model neparametrične regresije, ki nam omogoča ekstrapolacijo v naslednja leta. Opazimo, da se negotovost (varianca) napovedi povečuje, s tem ko se oddaljujemo od podatkov.

### Kovariančne funkcije

Kovariačne funkcije bistveno vplivajo na <i>obliko družine funkcij</i>, ki jih vzorčimo iz Gaussovega Procesa. Oglejmo si nekaj tipičnih primerov kovariačnih funkcij. Bodite pozorni na lastnosti družine funkcij.

In [15]:
kernels = [ GPy.kern.Linear, GPy.kern.RBF, GPy.kern.Brownian, GPy.kern.PeriodicExponential, GPy.kern.Poly]
names = ["linear", "exp", "brownian", "per_exp", "poly"]

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(14, 4)) 
for i, k, name in zip(range(5), kernels, names):
    
    # Narisi funkcijo kovariance
    knl = k(input_dim=1)
    knl.plot(x=1, ax=axes[0][i])
    axes[0][i].set_xlabel("t, t'")
    axes[0][i].set_ylabel("k(t, t')")
    axes[0][i].set_title(name)
    
    # Narisi vzorce iz druzine funkcij
    X = np.linspace(0, 10, 100).reshape((100, 1))
    mu = np.zeros((100, ))
    C = knl.K(X,X) 
    Z = np.random.multivariate_normal(mu,C,5)
    for z in Z:
        axes[1][i].plot(z)
    
fig.tight_layout()
plt.show()

NameError: name 'GPy' is not defined

Oglejmo si nekoliko bolj zanimiv signal. Spodnji podatki prikazujejo koncentracijo ogljikovega diosksida ($CO_2$) v ozračju od leta 1960.

In [16]:
co2 = np.genfromtxt("data/co2.csv", delimiter=",", skip_header=1)


n = len(co2)

t = co2[:, 0].reshape((n, 1))
x = co2[:, 2].reshape((n, 1)) 
x = x - x.mean()
co2

NameError: name 'np' is not defined

Opazimo sezonsko periodično spreminjanje signala, v kombinaciji z naraščajočim trendom.

In [17]:
plt.figure(figsize=(10, 4))
plt.plot(t, x, ".")
plt.ylabel("$CO_2$")
plt.xlabel("t (mesec)")
plt.show()

NameError: name 'plt' is not defined

<font color="green"><b>Naredi sam/a.</b></font> S seštevanjem kovariančnih funkcij poizkušaj modelirati podatke o koncetraciji $CO_2$. Poizkusi najto kombinacijo funkcij, ki najbolje ekstrapolirajo koncentracijo $CO_2$ v prihodnja leta.

In [18]:
kernel = GPy.kern.RBF(1, lengthscale=1)
model  = GPy.models.GPRegression(t, x, kernel, noise_var=0.1)
model.optimize(messages=True)
model.plot()
plt.gca().set_xlim(200, 600)

NameError: name 'GPy' is not defined